# Imports

In [1]:
import plotly.express as px
import pandas as pd

# Grab Data

In [2]:
df_rec = pd.read_pickle("./wr-simple-data-2012-2022.pkl")

In [3]:
df_rec["yac_per_reception"] = df_rec["receiving_yards_after_catch"] / df_rec["receptions"]

In [4]:
df_wr = df_rec.query("position == 'WR'").copy()

In [5]:
df_te = df_rec.query("position == 'TE'").copy()

In [6]:
df_rb = df_rec.query("position == 'RB'").copy()

# Normalization

In [7]:
def normalize_stats(df, stats_to_normalize, new_metric_components, group_by=["age"], new_metric="trinity"):
    """"""
    agg_dict = {}
    stats_to_normalize += ["fantasy_points"]
    for stat in stats_to_normalize:
        stat_norm = f"{stat}_normalized"
        df[stat_norm] = df[stat] / df[stat].max()
        
        agg_dict[stat_norm] = ["mean"]
        
    agg_dict["player_name"] = ["count"]
    
    df_norm = df \
            .groupby(group_by) \
            .agg(agg_dict)
    
    df_norm.columns = list(map("_".join, df_norm.columns))
    df_norm.reset_index(inplace=True)
    
    df_norm.rename(columns={"player_name_count": "sample_size"}, inplace=True)
    
    x = 0
    for col in new_metric_components:
        x += df_norm[col]
        
    df_norm[new_metric] = x / len(new_metric_components)
    
    return df_norm

In [8]:
query = "games >= 10"
if query:
    df_to_norm = df_wr.query(query).copy()
else:
    df_to_norm = df_wr.copy()

# Temp

In [ ]:
import itertools
import numpy as np

def get_all_col_combinations(df):
    # List to store all combinations
    all_combinations = []
    
#     numeric_columns = ['receptions', 'targets', 'receiving_yards', 'receiving_tds',
#        'receiving_air_yards',
#        'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
#        'racr', 'target_share', 'air_yards_share',
#        'wopr_x', 'tgt_sh',
#        'ay_sh', 'yac_sh', 'wopr_y', 'ry_sh', 'rtd_sh', 'rfd_sh', 'rtdfd_sh',
#        'dom', 'w8dom', 'yptmpa', 'ppr_sh', 'age', 'yac_per_reception']
    
    numeric_columns = ['receptions', 'targets', 'receiving_yards',
       'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs',
       'racr',
       'wopr_x', 'tgt_sh',
       'ay_sh', 'yac_sh', 'wopr_y', 'ry_sh', 'rfd_sh',
       'dom', 'w8dom', 'yptmpa', 'ppr_sh', 'yac_per_reception']
    
    # Generate combinations of all lengths
    for r in range(1, len(numeric_columns) + 1):
        for combination in itertools.combinations(numeric_columns, r):
            all_combinations.append(combination)
            
    return all_combinations

In [ ]:
combinations = get_all_col_combinations(df_to_norm)

In [ ]:
def loop_for_best_corrs(df_to_norm, combinations, x="age"):
    record = {}

    for combo in combinations:
        stats_to_normalize = list(combo)
        
        new_metric_components = []
        for stat in stats_to_normalize:
            new_metric_components.append(f"{stat}_normalized_mean")

        df_graph = normalize_stats(df_to_norm, stats_to_normalize, new_metric_components, x)

        query = "sample_size >= 10"
        if query:
            df_graph = df_graph.query(query)

        df_corr = pd.DataFrame(columns=["Stat", "Correlation-with-Fantasy-Points"])
        for col in list(df_graph.columns):
            if "age" not in col and "season" not in col and "fantasy" not in col and "sample_size" not in col:
                temp_corr_mat_fp = df_graph[[col, "fantasy_points_normalized_mean"]].corr()
                corr_value_fp = temp_corr_mat_fp[col].iloc[1]

                row = (col, corr_value_fp)
                row_df = pd.DataFrame([row], columns=df_corr.columns)
                df_corr = pd.concat([df_corr, row_df], ignore_index=True)

        trinity_corr_value = round(df_corr.query("Stat == 'trinity'")["Correlation-with-Fantasy-Points"].iloc[0], 3)
        record[trinity_corr_value] = new_metric_components
    
    return record

In [82]:
record = loop_for_best_corrs(df_to_norm, combinations[100:1000], "age")

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

...

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/403115015.py:25: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

In [83]:
sorted_record = {k: record[k] for k in sorted(record, reverse=True)}

In [84]:
list(sorted_record.items())[:3]

[(0.991,
  ['receiving_air_yards_normalized_mean',
   'receiving_yards_after_catch_normalized_mean',
   'dom_normalized_mean']),
 (0.983,
  ['receiving_yards_normalized_mean',
   'dom_normalized_mean',
   'w8dom_normalized_mean']),
 (0.982,
  ['receiving_air_yards_normalized_mean',
   'yac_sh_normalized_mean',
   'dom_normalized_mean'])]

# Temp

In [81]:
['tgt_sh', 'ay_sh', 'yac_per_reception']  # 0.844 trinity to fp
['wopr_y', 'rfd_sh', 'tgt_sh', 'ay_sh', 'yac_per_reception'] # 0.877 trinity to fp
# (0.985, ['receiving_yards_normalized_mean', 'dom_normalized_mean'])
# (0.963, ['receiving_yards_normalized_mean', 'w8dom_normalized_mean'])
# (0.991, ['receiving_air_yards_normalized_mean', 'receiving_yards_after_catch_normalized_mean', 'dom_normalized_mean']),
# (0.983, ['receiving_yards_normalized_mean', 'dom_normalized_mean', 'w8dom_normalized_mean']),
# (0.982, ['receiving_air_yards_normalized_mean', 'yac_sh_normalized_mean', 'dom_normalized_mean'])]

['wopr_y', 'rfd_sh', 'tgt_sh', 'ay_sh', 'yac_per_reception']

In [85]:
stats_to_normalize = ['receiving_air_yards', 'receiving_yards_after_catch', 'dom']
new_metric_components = []
for stat in stats_to_normalize:
    new_metric_components.append(f"{stat}_normalized_mean")

In [86]:
x = "age"

In [87]:
df_graph = normalize_stats(df_to_norm, stats_to_normalize, new_metric_components, x)

query = "sample_size >= 10"
if query:
    df_graph = df_graph.query(query)

# Line Graphs

In [88]:
base_path = "../../interactive/WR/stability/season"

In [89]:
fig = px.line(df_graph, x=x,
              y = new_metric_components + ["fantasy_points_normalized_mean"] + ["trinity"],
              hover_data=["sample_size"]
             )

# Update layout
fig.update_layout(
    title=f'Performance Metrics vs {x.capitalize()}',
    xaxis_title=x.capitalize(),
    yaxis_title='Performance Metrics',
    legend_title='Pefromance Metrics',
)

# save the figure
# fig.write_html(f"{base_path}/trinity-starters-by-age.html")

# Show the figure
fig.show()

In [90]:
df_corr = pd.DataFrame(columns=["Stat", "Correlation-with-Fantasy-Points"])
for col in list(df_graph.columns):
    if "age" not in col and "season" not in col and "fantasy" not in col and "sample_size" not in col:
        temp_corr_mat_fp = df_graph[[col, "fantasy_points_normalized_mean"]].corr()
        corr_value_fp = temp_corr_mat_fp[col].iloc[1]

        row = (col, corr_value_fp)
        row_df = pd.DataFrame([row], columns=df_corr.columns)
        df_corr = pd.concat([df_corr, row_df], ignore_index=True)

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_61198/3180196241.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [91]:
df_corr

,Stat,Correlation-with-Fantasy-Points
0,receiving_air_yards_normalized_mean,0.844329
1,receiving_yards_after_catch_normalized_mean,0.704646
2,dom_normalized_mean,0.922924
3,trinity,0.991005


In [ ]:
fig = px.bar(df_corr.sort_values(by="Correlation-with-Fantasy-Points", ascending=False), x="Stat", 
             y=["Correlation-with-Fantasy-Points"])

# Update layout
fig.update_layout(
    title='Stat Correlation with Fantasy Points',
    xaxis_title='Stat',
    yaxis_title='Correlation Value',
    legend_title='Correlation Type'
)

# fig.write_html(f"{base_path}/trinity-corr-by-{x}.html")

fig.show()

# Scatter

In [ ]:
def normalize_stats_scatter(df):
    """"""
    df["fantasy_points_per_game"] = df["fantasy_points"] / df["games"]
    
    df["fantasy_points_normalized"] = df["fantasy_points"] / df["fantasy_points"].max()
    df["fantasy_points_per_game_normalized"] = df["fantasy_points_per_game"] / df["fantasy_points_per_game"].max()
    df['yac_normalized'] = df['receiving_yards_after_catch'] / df['receiving_yards_after_catch'].max()
    df['ray_normalized'] = df["receiving_air_yards"] / df["receiving_air_yards"].max()
    
    df["trinity_normalized"] = (df['yac_normalized'] + df["ray_normalized"] + df["tgt_sh"]) / 3
    
    return df

In [ ]:
df_scatter = normalize_stats_scatter(df_to_norm)

In [ ]:
df_scatter \
    .query("season == 2022 and games >= 10")[["player_name", "trinity_normalized", "fantasy_points_normalized"]] \
    .sort_values(by="trinity_normalized", ascending=False) \
    .head(1)

In [ ]:
df_scatter.head()

In [ ]:
fig = px.scatter(df_scatter, x='trinity_normalized',
                 y='fantasy_points_normalized',
                 hover_data=["player_name", "season", "age"],
             )

# Update layout
fig.update_layout(
    title='Fantasy Points Normalzied vs. Trinity Normalized',
    xaxis_title='Trinity Normalized',
    yaxis_title='Fantasy Points Normalized',
)

# save the figure
# fig.write_html(f"{base_path}-10-games-10-samples-carries.html")

# Show the figure
fig.show()

In [ ]:
fig = px.scatter(df_scatter, x='trinity_normalized',
                 y='fantasy_points_normalized',
                 hover_data=["player_name", "season", "age"],
                 color="season"
             )

# Update layout
fig.update_layout(
    title='Fantasy Points Normalzied vs. Trinity Normalized',
    xaxis_title='Trinity Normalized',
    yaxis_title='Fantasy Points Normalized',
)

# save the figure
# fig.write_html(f"{base_path}-10-games-10-samples-carries.html")

# Show the figure
fig.show()

In [ ]:
fig = px.scatter(df_scatter, x='trinity_normalized',
                 y='fantasy_points_normalized',
                 hover_data=["player_name", "season", "age"],
                 color="age"
             )

# Update layout
fig.update_layout(
    title='Fantasy Points Normalzied vs. Trinity Normalized',
    xaxis_title='Trinity Normalized',
    yaxis_title='Fantasy Points Normalized',
)

# save the figure
# fig.write_html(f"{base_path}-10-games-10-samples-carries.html")

# Show the figure
fig.show()

In [ ]:
fig = px.scatter(df_scatter.query("age <= 30"), x='trinity_normalized',
                 y='fantasy_points_normalized',
                 hover_data=["player_name", "season", "age"],
                 color="age"
             )

# Update layout
fig.update_layout(
    title='Fantasy Points Normalzied vs. Trinity Normalized',
    xaxis_title='Trinity Normalized',
    yaxis_title='Fantasy Points Normalized',
)

# save the figure
# fig.write_html(f"{base_path}-10-games-10-samples-carries.html")

# Show the figure
fig.show()

# Players on Trend

In [ ]:
import plotly.graph_objects as go

In [ ]:
def player_v_trend_trinity(df, df_graph, players):
    """"""
    fig = px.line(df_graph, x='age',
                      y=['fantasy_points_normalized_mean', "trinity"],
                      hover_data=["sample_size"]
                 )
    
    df_normalized = normalize_stats(df, ["player_id", "season", "player_name", "age"])
    
    for player_name in players:
        df_player = df_normalized.query(f"player_name == '{player_name}'")
        fig.add_trace(
            go.Scatter(x=df_player['age'], y=df_player["trinity"],
                       mode='lines', name=f"{player_name} trinity")
        )

        
    # Update layout
    fig.update_layout(
        title='Performance Metrics vs Age',
        xaxis_title='Age',
        yaxis_title='Performance Metrics',
        legend_title='Pefromance Metrics',
    )

    # save the figure
    # fig.write_html(f"{base_path}-10-games-10-samples-carries.html")

    # Show the figure
    fig.show()

In [ ]:
player_v_trend_trinity(df_wr, df_graph, ["Keenan Allen", "Mike Evans", "Adam Thielen", "DeAndre Hopkins",
                                        "Cooper Kupp", "Antonio Brown"])

In [ ]:
fig = px.line(df_graph, x='age',
              y=['fantasy_points_normalized_mean', "trinity"],
              hover_data=["sample_size"]
             )

# Add a line from another DataFrame
fig.add_trace(
    go.Scatter(x=p['age'], y=p["trinity"],
               mode='lines', name='Sun God')
)

# Add a line from another DataFrame
fig.add_trace(
    go.Scatter(x=p1['age'], y=p1["trinity"],
               mode='lines', name='Pittman')
)

# Add a line from another DataFrame
fig.add_trace(
    go.Scatter(x=p2['age'], y=p2["trinity"],
               mode='lines', name='Evans')
)

# Add a line from another DataFrame
fig.add_trace(
    go.Scatter(x=p3['age'], y=p3["trinity"],
               mode='lines', name='Brown')
)

# Add a line from another DataFrame
fig.add_trace(
    go.Scatter(x=p4['age'], y=p4['trinity'],
               mode='lines', name='Allen trin')
)
fig.add_trace(
    go.Scatter(x=p4['age'], y=p4['fantasy_points_normalized_mean'],
               mode='lines', name='Allen fp')
)

fig.add_trace(
    go.Scatter(x=p5['age'], y=p5['trinity'],
               mode='lines', name='Aiyuk trin')
)
fig.add_trace(
    go.Scatter(x=p5['age'], y=p5['fantasy_points_normalized_mean'],
               mode='lines', name='Aiyuk fp')
)

# Update layout
fig.update_layout(
    title='Performance Metrics vs Age',
    xaxis_title='Age',
    yaxis_title='Performance Metrics',
    legend_title='Pefromance Metrics',
)

# save the figure
# fig.write_html(f"{base_path}-10-games-10-samples-carries.html")

# Show the figure
fig.show()